In [ ]:
import pandas as pd

df_train = pd.read_csv('ntrain.csv')
df_train.head()

In [ ]:
#Ftiaxnei ta dunamosunola
def powerset(seq):
    #counter = 0
    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for item in powerset(seq[1:]):
            yield [seq[0]]+item
            yield item
            #couter += 1
    #print(counter)


In [ ]:
l = [0, 1, 2, 3, 4, 5, 6, 7]
r = [x for x in powerset(l)]
r.sort()
print(r)
print(r[0])

In [ ]:
import numpy as np

def whoTodrop(i):
    columns=[];
    for k in range(8):
           for j in range(len(r[i])):
                if(k!=r[i][j]):
                    columns.append(k)
    return columns 

In [ ]:
df_test = pd.read_csv('ntest.csv')
y_train = df_train[['PAX']]
from sklearn.model_selection import train_test_split
df_train, df_test, y_train, y_test = train_test_split(df_train, y_train, test_size=0.2, random_state=42)
y_train = np.ravel(y_train)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_train['Departure'])
df_train['Departure'] = le.transform(df_train['Departure'])
df_train['Arrival'] = le.transform(df_train['Arrival'])
df_test['Departure'] = le.transform(df_test['Departure'])
df_test['Arrival'] = le.transform(df_test['Arrival'])
le.fit(df_train['CityDeparture'])
df_train['CityDeparture'] = le.transform(df_train['CityDeparture'])
df_train['CityArrival'] = le.transform(df_train['CityArrival'])
df_test['CityDeparture'] = le.transform(df_test['CityDeparture'])
df_test['CityArrival'] = le.transform(df_test['CityArrival'])
le = LabelEncoder()
le.fit(df_train['DateOfDeparture'])
df_train['DateOfDeparture'] = le.transform(df_train['DateOfDeparture'])
df_test['DateOfDeparture'] = le.transform(df_test['DateOfDeparture'])
print(df_train)

In [ ]:
#TEST
df_train.columns.get_loc('Departure')

In [ ]:
def findCateg(i):
    catf =[];
    t0 = 0
    t1 = 0
    t2 = 0
    t3 = 0
    t4 = 0
    for k in range(len(r[i])):
        if(r[i][k]==1):
            t1 = 1
        elif(r[i][k]==2):
            t2 = 1
        elif(r[i][k]==3):
            t3 = 1
        elif(r[i][k]==4):
            t4 = 1
        elif(r[i][k]==0):
            t0 = 1
    if(t0==0):
        ind0=dfTr.columns.get_loc('DateOfDeparture')
        catf.append(ind0)
    if(t1==0):
        ind0=dfTr.columns.get_loc('Departure')
        catf.append(ind0)
    if(t2==0):
        ind0=dfTr.columns.get_loc('CityDeparture')
        catf.append(ind0)
    if(t3==0):
        ind0=dfTr.columns.get_loc('Arrival')
        catf.append(ind0)
    if(t4==0):
        ind0=dfTr.columns.get_loc('CityArrival')
        catf.append(ind0)
    return catf

In [ ]:
#NEW ENTRY
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
#from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import numpy as np
from sklearn.metrics import f1_score

df_train.drop(df_train.columns[[3,4,7,8,12]], axis=1, inplace=True)
df_test.drop(df_test.columns[[3,4,7,8,12]], axis=1, inplace=True)
print(df_train.columns[3])
score=[]
r.sort()
k=3
for i in range(len(r)):
    if(len(r[i])<8):
        dfTr = df_train.copy()
        dfTe= df_test.copy()
        col=whoTodrop(i)
        #print(dfTr)
        #print(df_train)
        #print(col)
        print(r[i])
        dfTr.drop(dfTr.columns[r[i]], axis=1, inplace=True)
        print(dfTr)
        print("DF_TRAIN")
        print(df_train)
        dfTe.drop(dfTe.columns[r[i]], axis=1, inplace=True)
        cat_f = findCateg(i)
        if cat_f:
            print('in onehot')
            enc = OneHotEncoder(categorical_features=cat_f,sparse=False)#check argument
            enc.fit(dfTr)
            X_train = enc.transform(dfTr)
            X_test = enc.transform(dfTe)
        else:
            X_train = dfTr
            X_test = dfTe

        #X_train = dfTr
        #X_test = dfTe
        #print(X_test)
        #clf = LogisticRegression()
        clf = KNeighborsClassifier(k)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scr=f1_score(y_test, y_pred, average='micro')
        print(i,scr)
        score.append(scr)
    elif(i==0):
        dfTr = df_train.copy()
        dfTe= df_test.copy()
        #col=whoTodrop(i)
        cat_f = [0,1,2,3,4]
        enc = OneHotEncoder(categorical_features=cat_f,sparse=False)#check argument
        enc.fit(dfTr)
        X_train = enc.transform(dfTr)
        X_test = enc.transform(dfTe)        
        #X_train = dfTr
        #X_test = dfTe
        #print(X_test)
        clf = LogisticRegression(solver='lbfgs', multi_class='multinomial')
        #clf = KNeighborsClassifier(k)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scr=f1_score(y_test, y_pred, average='micro')
        print(i,scr)
        score.append(scr)
    elif(len(r[i])==8):
        score.append(0)

In [ ]:
print(len(r))

In [ ]:
import csv
def calculate_dynamic_set(num_columns):
    
    with open('regression_resultslr', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['Set to drop', 'Result'])
        for i in range(len(r)):
            writer.writerow([r[i], score[i]])

In [ ]:
calculate_dynamic_set(8)